# Train mô hình Logistic Regression

In [4]:
import numpy as np
import random

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

In [5]:
import os
import json
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd


## Prepare Data

In [6]:
def prepareData(dir_train, dir_test):
    data_train = pd.read_csv(dir_train)
    data_test = pd.read_csv(dir_test)

    X_train = data_train.drop(['Activity', 'Activity_code'], axis=1)
    y_train = data_train['Activity_code']

    X_test = data_test.drop(['Activity', 'Activity_code'], axis=1)
    y_test = data_test['Activity_code']

    return X_train, X_test, y_train, y_test


## Load Best Hyperparameter

In [7]:
def load_parameters_from_json(json_path):
    if not os.path.exists(json_path):
        raise FileNotFoundError(f"File {json_path} không tồn tại!")
    with open(json_path, 'r', encoding='utf-8') as file:
        params = json.load(file)
    return params


## Create train Model function

In [8]:
def train_logistic_regression(X_train, y_train, X_test, y_test, params):

    C = params['C']
    penalty = params['penalty']
    solver = params['solver']
    max_iter = params['max_iter']

    model = LogisticRegression(penalty=penalty, C=C, solver=solver, max_iter=max_iter)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy}")
    return model

## Save Model function

In [9]:
def save_model(model, model_path):
    joblib.dump(model, model_path)
    print(f"Mô hình đã được lưu tại: {model_path}")

## Train model

In [12]:
dir_train = "../../data/interim/trainFinal.csv"
dir_test = "../../data/interim/testFinal.csv"
dir_param = "../hyperparameter_tuning/Best_Hyperparameter/best_parameter_LogisticRegression.json"
params = load_parameters_from_json(dir_param)
X_train, X_test, y_train, y_test = prepareData(dir_train, dir_test)
X_train = X_train.values
X_test = X_test.values

In [13]:
model = train_logistic_regression(X_train, y_train, X_test, y_test, params)

Accuracy: 0.9616559212758737


In [ ]:
save_model(model, "../../models/LogisticRegression_model.pkl")

## Train model với dữ liệu PCA

In [15]:
dir_train = "../../data/interim/train_PCA.csv"
dir_test = "../../data/interim/test_PCA.csv"

data_train_PCA = pd.read_csv(dir_train)
data_test_PCA = pd.read_csv(dir_test)

model_PCA = train_logistic_regression(data_train_PCA, y_train, data_test_PCA, y_test, params)

Accuracy: 0.9304377332880895
